In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)

#Seaborn is a library for making statistical graphics in Python.
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

from datetime import timedelta
import datetime as dt
from datetime import datetime
import calendar

from collections import Counter

In [2]:
train =  pd.read_csv('train.csv',sep =',', nrows = 2000000)

In [3]:
train.columns

Index(['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count'],
      dtype='object')

In [4]:
test = pd.read_csv('test.csv',sep =',')

In [5]:
train.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,2.000000e+06,2.000000e+06,2.000000e+06,1.999986e+06,1.999986e+06,2.000000e+06
mean,1.134779e+01,-7.252321e+01,3.992963e+01,-7.252395e+01,3.992808e+01,1.684113e+00
std,9.852883e+00,1.286804e+01,7.983352e+00,1.277497e+01,1.032382e+01,1.314982e+00
min,-6.200000e+01,-3.377681e+03,-3.458665e+03,-3.383297e+03,-3.461541e+03,0.000000e+00
25%,6.000000e+00,-7.399208e+01,4.073491e+01,-7.399141e+01,4.073400e+01,1.000000e+00
50%,8.500000e+00,-7.398181e+01,4.075263e+01,-7.398016e+01,4.075312e+01,1.000000e+00
75%,1.250000e+01,-7.396713e+01,4.076710e+01,-7.396369e+01,4.076809e+01,2.000000e+00
max,1.273310e+03,2.856442e+03,2.621628e+03,3.414307e+03,3.345917e+03,2.080000e+02


In [6]:
# define rectangular region in latitude and longitude scale
BB = (-74.5, -72.8, 40.5, 41.8)

In [7]:
def select_bdry(df, BB):
    return (df.pickup_longitude >= BB[0]) & (df.pickup_longitude <= BB[1]) & \
           (df.pickup_latitude >= BB[2]) & (df.pickup_latitude <= BB[3]) & \
           (df.dropoff_longitude >= BB[0]) & (df.dropoff_longitude <= BB[1]) & \
           (df.dropoff_latitude >= BB[2]) & (df.dropoff_latitude <= BB[3])

In [8]:
train = train[select_bdry(train, BB)]

In [12]:
def remove_datapoints_from_water(df):
    def lonlat_to_xy(longitude, latitude, dx, dy, BB):
        return (dx*(longitude - BB[0])/(BB[1]-BB[0])).astype('int'), (dy - dy*(latitude - BB[2])/(BB[3]-BB[2])).astype('int')

    # define rectangular region
    BB = (-74.5, -72.8, 40.5, 41.8)
    
    # read nyc mask and turn into boolean map with
    nyc_mask = plt.imread('nyc_mask-74.5_-72.8_40.5_41.8.png')[:,:,0] > 0.9
    
    # calculate for each lon,lat coordinate the xy coordinate in the mask map
    pickup_x, pickup_y = lonlat_to_xy(df.pickup_longitude, df.pickup_latitude, 
                                      nyc_mask.shape[1], nyc_mask.shape[0], BB)
    dropoff_x, dropoff_y = lonlat_to_xy(df.dropoff_longitude, df.dropoff_latitude, 
                                      nyc_mask.shape[1], nyc_mask.shape[0], BB)    
    # calculate boolean index
    idx = nyc_mask[pickup_y, pickup_x] & nyc_mask[dropoff_y, dropoff_x]
    
    # return only datapoints on land
    return df[idx]

In [13]:
train.columns

Index(['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count'],
      dtype='object')

In [14]:
train = remove_datapoints_from_water(train)

In [15]:
def clean_data(df):
    # drop key
    #df = df.drop(['key'], axis = 1)
    area = [-74.5, -72.8, 40.5, 41.8]
    
    # df without missing values
    df = df.dropna(how = 'any', axis = 'rows')  
    
    # choosing latitude and longitude within the area
    df = df[select_bdry(df, area)]
    
    # remove points that have location on the sea
    df = remove_datapoints_from_water(df)
    
    # fare amount should be (0,200]
    df = df[(df['fare_amount'] > 0) & (df['fare_amount'] <= 200)]
    #df = df[df['fare_amount'].between(left = 2.5, right = 100)]
    
    # passenger_count should be [0,6]
    df = df[(df['passenger_count'] >= 0) & (df['passenger_count'] <= 6)]
    return df

In [16]:
train.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,1.957600e+06,1.957600e+06,1.957600e+06,1.957600e+06,1.957600e+06,1.957600e+06
mean,1.132839e+01,-7.397516e+01,4.075109e+01,-7.397428e+01,4.075143e+01,1.684216e+00
std,9.714251e+00,3.847423e-02,2.952658e-02,3.764636e-02,3.269997e-02,1.306688e+00
min,-6.200000e+01,-7.448963e+01,4.050005e+01,-7.449105e+01,4.050005e+01,0.000000e+00
25%,6.000000e+00,-7.399229e+01,4.073655e+01,-7.399159e+01,4.073554e+01,1.000000e+00
50%,8.500000e+00,-7.398210e+01,4.075335e+01,-7.398062e+01,4.075384e+01,1.000000e+00
75%,1.250000e+01,-7.396834e+01,4.076752e+01,-7.396537e+01,4.076839e+01,2.000000e+00
max,5.000000e+02,-7.281783e+01,4.169685e+01,-7.281783e+01,4.171463e+01,9.000000e+00


In [17]:
test = pd.read_csv('test.csv',sep =',')

In [18]:
train = clean_data(train)